<a href="https://colab.research.google.com/github/Polineska/SpeechRecognition/blob/main/vosk_analysisGolos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!sudo apt update && sudo apt install ffmpeg
!pip install jiwer -q

In [ ]:
!pip3 install vosk

In [3]:
import sys
sys.executable
import os
import warnings
import jiwer
import torch
from datasets import load_dataset
import numpy as np

In [14]:
#bond005/sberdevices_golos_10h_crowd_noised_2db #для зашумленных аудиоданных
DATASET_ID = "bond005/sberdevices_golos_10h_crowd" #для незашумленных
SAMPLES = 9900

test_dataset = load_dataset(DATASET_ID, split=f"test[:{SAMPLES}]")



In [6]:
from vosk import Model, KaldiRecognizer
import wave
import json
import vosk
import jiwer
import pandas as pd
import librosa
import re

In [15]:
def speech_file_to_array_fn(batch):
    audio_file = batch["audio"].get("path")
    if audio_file is not None:
        try:
            speech_array, sampling_rate = librosa.load(audio_file, sr=16000)
            speech_array = speech_array.astype(np.int16)
            batch["speech"] = speech_array
        except Exception as e:
            print(f"Error loading file {audio_file}: {e}")
    return batch

# Предобработка датасета
removed_columns = set(test_dataset.column_names)
removed_columns -= {'transcription', 'speech'}
removed_columns = sorted(list(removed_columns))
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    test_dataset = test_dataset.map(
        speech_file_to_array_fn,
        remove_columns=removed_columns
    )

def infer_vosk(samples): # Функция для распознавания речи с помощью Vosk
    transcripts = []
    model = Model(lang='ru')
    for i, sample in enumerate(samples):
        if "speech" in sample:
            audio = sample["speech"]
            rec = KaldiRecognizer(model, 16000)
            rec.SetMaxAlternatives(10)
            rec.SetPartialWords(True)

            # Распознавание речи
            rec.AcceptWaveform(audio.tobytes())
            result = rec.FinalResult()

            # Получение текста распознавания
            if isinstance(result, dict):
                alternatives = result.get_alternatives()
                if alternatives:
                    transcript_text = alternatives[0].get('text', '')
                else:
                  transcript_text = ''
                  transcripts.append(transcript_text)

            return transcripts
hypotheses = infer_vosk(test_dataset)
references = test_dataset["transcription"]

In [18]:
def infer_vosk(samples):
    transcripts = []
    model = Model(lang='ru')
    for sample in samples:
        if "speech" in sample:
            audio = sample["speech"]
            rec = KaldiRecognizer(model, 16000)
            rec.SetMaxAlternatives(10)
            rec.SetPartialWords(True)

            # Распознавание речи
            rec.AcceptWaveform(audio.tobytes())
            result = rec.FinalResult()

            # Получение текста распознавания
            if result:
                if isinstance(result, dict):
                    alternatives = result.get('alternatives', [])
                else:
                    alternatives = result.alternatives
                if alternatives:
                    transcript_text = alternatives[0]['text']
                    transcripts.append(transcript_text)
    return transcripts
hypotheses = infer_vosk(test_dataset)
references = test_dataset["transcription"]

In [8]:
df = pd.DataFrame(dict(hypothesis_small=hypotheses, reference_small=references))

In [22]:
def normalize_text(text):
    if not isinstance(text, str):
        text = str(text)

    # Удаление знаков пунктуации и преобразование в нижний регистр
    text = re.sub(r'[^\w\s]', '', text).lower()

    # Удаление множественных пробелов
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

refs = test_dataset["transcription"]
normalized_refs = [normalize_text(ref) for ref in refs]
normalized_hyps = [normalize_text(hyp) if hyp else '' for hyp in hypotheses]
max_length = max(len(normalized_refs), len(normalized_hyps))
normalized_refs += ['' for _ in range(max_length - len(normalized_refs))]
normalized_hyps += ['' for _ in range(max_length - len(normalized_hyps))]
wer_score = jiwer.wer(normalized_refs, normalized_hyps)
print(f"\nWER crowd_noised: {wer_score * 100:.2f}%")


WER crowd_noised: 87.50%


In [12]:
wer_score = jiwer.wer(normalized_refs, normalized_hyps)
print(f"\nWER crowd: {wer_score * 100:.2f}%")


WER crowd: 19.42%
